# Leaderboards Analysis

Based on our current scouting, as well as The Blue Alliance's OPR score, and any other criteria we have, create a ranked list of teams within the competition.

*run all Data notebooks before any of the analysis notebooks.*

In [21]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, HTMLTemplateFormatter
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Leaderboard_Analysis')

from common import canalytics
from common import CLIMB_POINTS,CLIMB_NAMES,FIRST_AVG,LAST_AVG,TOTAL_AVG


Loading BokehJS ...

In [22]:
team_summary = pd.read_csv('./data/team_summary.csv', index_col=0)


# Let's look at the number of matches we have scouting data for.
# If there are more than FIRST_AVG + LAST_AVG, let's calculate a scouting score for the first 3 matches, and the most recent 3 matches.
# This will allow us to see if they had early issues that they resolved that might mean their OPR/Total event scouting score is not representative.
# Likewise if the most recent score was lower, then they may have some reliability issues with the robot.
# Also, determine the highest climb position.
team_summary.head()

,team,matches,scouted_matches,scouting_points_avg,first_avg,last_avg,climb_success_pct,climb_failure_pct,total_cargo_avg,auto_cargo_avg,teleop_cargo_avg,climb_points_avg,highest_endgame_position,oprs,pre_oprs,pre_climb_points,pre_district_rank,pre_district_points
0,457,12,8,5.000000,3.333333,3.000000,16.666667,37.500000,1.250000,0.125,1.125000,1.000000,Mid,10.118089,10.12,10.83,84.0,24.0
1,2158,12,10,29.600000,16.666667,22.000000,50.000000,10.000000,10.500000,1.700,8.800000,3.000000,Mid,27.186538,27.19,8.25,55.0,33.0
2,2468,12,11,27.181818,23.666667,29.666667,91.666667,0.000000,7.818182,1.000,6.818182,10.916667,Traversal,29.282110,NaN,NaN,NaN,NaN
3,2583,12,6,1.500000,0.333333,1.000000,8.333333,16.666667,0.833333,0.000,0.833333,0.500000,Mid,7.418845,NaN,NaN,NaN,NaN
4,2687,12,10,25.000000,22.333333,15.000000,91.666667,0.000000,6.500000,0.700,5.800000,9.000000,Traversal,31.423240,NaN,NaN,NaN,NaN


In [26]:
"""
#First AVG/Last AVG only works if all matches are scouted. TODO:  Need to fix that.

team_summary = matches.groupby('team').agg( matches = pd.NamedAgg(column='match', aggfunc='count'),
                                      scouting_points = pd.NamedAgg(column='scouting_points', aggfunc = 'mean' ),
                                      highest_climb_points = pd.NamedAgg(column='climb_points', aggfunc='max'),
                                      climb_success_pct = pd.NamedAgg(column = 'climb_points', aggfunc = lambda x: 100 * (sum(x.where(x==0,other=1))/x.size ) ),
                                      first_avg = pd.NamedAgg(column='scouting_points', aggfunc = lambda x:  sum(x.head(FIRST_AVG))/FIRST_AVG if x.size >= TOTAL_AVG else 0),
                                      first_few = pd.NamedAgg(column='scouting_points', aggfunc = lambda x: ",".join(str(i) for i in x.head(FIRST_AVG))),
                                      last_few = pd.NamedAgg(column='scouting_points', aggfunc = lambda x: ",".join(str(i) for i in x.tail(LAST_AVG))),
                                      last_avg = pd.NamedAgg(column='scouting_points', aggfunc = lambda x: sum(x.tail(LAST_AVG))/LAST_AVG if x.size >=TOTAL_AVG else 0))
                                                         

team_summary['highest_endgame_position']=app_team_summary.apply( lambda row: CLIMB_NAME[row.highest_climb_points], axis=1)

"""

teams=team_summary[['team','oprs','scouting_points_avg',"matches",'first_avg','last_avg',"highest_endgame_position","climb_success_pct"]]
teams=teams.round()

In [27]:
source = ColumnDataSource(teams)


gocans_template="""                
            <p style="color:<%= 
                  (function colorfromteam(){
                    if (team == 2158 ){
                      return('#FFCF46')}
                    }()) %>;
                background-color:<%=
                  (function backgroundfromteam(){
                    if (team == 2158 ){
                      return('black')}
                    }()) %>;
                    "> 
                <%= value %>
            </p>
            """
team_formatter =  HTMLTemplateFormatter(template=gocans_template)

columns = [
        TableColumn(field="team", title="Team",width=50, formatter=team_formatter),
        TableColumn(field="matches", title="Matches", width=75),
        TableColumn(field="highest_endgame_position", title="Max Climb"),
        TableColumn(field="climb_success_pct", title="Climb Success Pct", width=175),
        TableColumn(field="oprs", title="OPR from TBA", width=175),
        TableColumn(field="scouting_points_avg", title="Average Scouting Points", width=200),
        TableColumn(field="first_avg", title="Avg Scouting from first", width=200),
        TableColumn(field="last_avg", title="Avg Scouting from last", width=200),
    ]

data_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        reorderable=True)


# Leaderboard

The table below shows their Offensive Power Rating Score from TBA, as well as the Average Scouting Score of the matches so far.
If our scouting is accurate, and the OPR is higher than our Average Scouting Score, then OPR might be higher than their overall value as an alliance partner.
If the OPR is lower than our average scouting score, then they may be a good alliance pick that may not be noticed by other teams.

We also display the first few and last few scouting averages to indicate if there are changes in match performance.


In [28]:
show(data_table)